In [ ]:
import numpy as np

file_path1 = '/root/shuju/xunlianji2.csv'
file_path2 = '/root/shuju/ceshiji.csv'
data = np.genfromtxt(file_path1, delimiter=',', skip_header=1)
data_test = np.genfromtxt(file_path2, delimiter=',', skip_header=1)
np.save('/root/shuju/fangan3data2mev.npy', data)
np.save('/root/shuju/2015-10-12.npy', data_test)

In [ ]:
import numpy as np
import pandas as pd

input_file_path1 = '/root/shuju/fangan3data2mev.npy'  
input_file_path2 = '/root/shuju/2015-10-12.npy'

data1 = np.load(input_file_path1)
datax = data1[:,1:50]                      
datay = np.reshape(data1[:, 50],(-1,1))    
print(datax.shape)
print(datay.shape)

data3 = np.load(input_file_path2)
datax_test = data3[:,1:50]                     
datay_test = np.reshape(data3[:, 50],(-1,1))  


In [ ]:
datax = mm.fit_transform(datax)
datay = mm.fit_transform(datay)
datax_test = mm.fit_transform(datax_test)
datay_test = mm.fit_transform(datay_test)

In [ ]:
print(datax)
print(datay)

print(datax_test)
print(datay_test)

In [ ]:
x_data = np.array(datax)
y_data = np.array(datay)

x_test = np.array(datax_test)
y_test = np.array(datay_test)

In [ ]:
import tensorflow as tf
import keras.backend as K

def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

def pe(y_true, y_pred):
    return 1 - (K.sum(K.square(y_pred - y_true)) / K.sum(K.square(y_true - K.mean(y_true))))

def cc(y_true, y_pred):
    mean_true = K.mean(y_true)
    mean_pred = K.mean(y_pred)
    covar = K.mean((y_true - mean_true) * (y_pred - mean_pred))
    std_true = K.std(y_true)
    std_pred = K.std(y_pred)
    return covar / (std_true * std_pred)

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K

def custom_loss(y_true, y_pred):
    sigmoid_coefficient = (1 + (69.92719340161796 / (1 + tf.exp(-5.53710329830428 * (y_true - 3.504586735373479)))))
    loss = K.mean(K.square(y_pred - y_true) * sigmoid_coefficient)
    return loss

In [ ]:
num_hidden_1 = 42
activation_1 = 'relu'
reg_param_1 = 9.755667800692107e-05
regularizer_1 = tf.keras.regularizers.l1(reg_param_1)

model.add(layers.Dense(num_hidden_1, kernel_initializer='he_normal', activation=activation_1, kernel_regularizer=regularizer_1))

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.initializers import he_normal
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras import layers

def pred_model():
    model = tf.keras.Sequential()  
    model.add(layers.Dense(49, kernel_initializer='he_normal', activation='relu', input_shape=(49,)))
    num_hidden_1 = 61
    activation_1 = 'relu'
    reg_param_1 = 0.005
    regularizer_1 = tf.keras.regularizers.l2(reg_param_1)
    model.add(layers.Dense(num_hidden_1, kernel_initializer='he_normal', activation=activation_1, kernel_regularizer=regularizer_1))
    num_hidden_2 = 56
    activation_2 = 'relu'
    reg_param_2 = 0.001
    regularizer_2 = tf.keras.regularizers.l1(reg_param_2)    
    model.add(layers.Dense(num_hidden_2, kernel_initializer='he_normal', activation=activation_2, kernel_regularizer=regularizer_2))
    num_hidden_3 = 58
    activation_3 = 'sigmoid'
    reg_param_3 = 0.001
    regularizer_3 = tf.keras.regularizers.l2(reg_param_3)     
    model.add(layers.Dense(num_hidden_3, kernel_initializer='he_normal', activation=activation_3, kernel_regularizer=regularizer_3))
    model.add(layers.Dense(1, activation='linear'))
    return model

In [ ]:
model = pred_model()  
reduce_lr = ReduceLROnPlateau(monitor='val_loss',    
                             factor=0.7,            
                             patience=20,            
                             mode='auto',          
                             verbose=1)            

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=20, 
                               verbose=1, 
                               restore_best_weights=True)

In [ ]:

import numpy as np
from sklearn.model_selection import KFold
from keras.callbacks import ModelCheckpoint

group_size = 288
num_groups = len(x_data) // group_size
num_folds = 12
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
all_train_losses = []
all_val_losses = []
k_fold_pe_values = []
weights_list = []
with open('/root/shuju/training_process.txt', 'w') as file:
    for fold, (train_groups, val_groups) in enumerate(kf.split(range(num_groups))):
        file.write(f"在第 {fold + 1}/{num_folds} 折上进行训练\n")
        train_indices = [idx for group_idx in train_groups for idx in range(group_idx * group_size, (group_idx + 1) * group_size)]
        val_indices = [idx for group_idx in val_groups for idx in range(group_idx * group_size, (group_idx + 1) * group_size)]
        x_train, y_train = x_data[train_indices], y_data[train_indices]
        x_val, y_val = x_data[val_indices], y_data[val_indices]
        model.compile(optimizer='adam', loss=custom_loss, metrics=[rmse, pe, cc])
        checkpoint_filepath = f"/root/shuju/fangan72mev_model_fold_{fold}.h5"
        model_checkpoint = ModelCheckpoint(checkpoint_filepath, 
                                            save_best_only=True, 
                                            save_weights_only=True, 
                                            monitor='val_loss', 
                                            mode='min',         
                                            verbose=1)    
        history = model.fit(x_train, 
                            y_train, 
                            epochs=87, 
                            batch_size=1200, 
                            validation_data=(x_val, y_val),
                            verbose=1, 
                            shuffle=True, 
                            callbacks=[reduce_lr, model_checkpoint,early_stopping])

        file.write(f"训练历史：{history.history['loss']}\n")
        file.write(f"验证历史：{history.history['val_loss']}\n")
        file.write(f"验证 PE 值：{history.history['val_pe']}\n")
        all_train_losses.append(history.history['loss'])
        all_val_losses.append(history.history['val_loss'])
        k_fold_pe_values.append(history.history['val_pe'])
        weights = model.get_weights()
        weights_list.append(weights)

average_weights = np.mean(weights_list, axis=0)
model.set_weights(average_weights)
average_weights_filepath = "/root/shuju/fangan72mev_average_weights.h5"
model.save_weights(average_weights_filepath)
print(f"平均权重已保存至 {average_weights_filepath}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

miniepoch = 20
all_train_losses = [epoch_data[:miniepoch] for epoch_data in all_train_losses]
all_val_losses = [epoch_data[:miniepoch] for epoch_data in all_val_losses]

all_train_losses = np.array(all_train_losses)
all_val_losses = np.array(all_val_losses)
mean_train_loss = np.mean(np.array(all_train_losses), axis=0)
mean_val_loss = np.mean(np.array(all_val_losses), axis=0)
std_train_loss = np.std(np.array(all_train_losses), axis=0)
std_val_loss = np.std(np.array(all_val_losses), axis=0)

c = range(miniepoch)
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(c, mean_train_loss, "b-", linestyle='solid', label="mean_train_loss")
ax.plot(c, mean_val_loss, "r-", linestyle='solid', label="mean_val_loss")
ax.fill_between(c, mean_train_loss - std_train_loss, mean_train_loss + std_train_loss, color='blue', alpha=0.3)
ax.fill_between(c, mean_val_loss - std_val_loss, mean_val_loss + std_val_loss, color='red', alpha=0.3)
ax.legend()
ax.set_xlabel('epoch')
ax.set_ylabel('value')
ax.set_title('2mev: Compare mean_train_loss and mean_val_loss with standard deviation')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

k_fold_pe_values = [epoch_data[:miniepoch] for epoch_data in k_fold_pe_values]
k_fold_pe_values = np.array(k_fold_pe_values)

mean_pe_values = np.mean(k_fold_pe_values, axis=0)
std_pe_values = np.std(k_fold_pe_values, axis=0)

epochs = range(1, len(mean_pe_values) + 1)

plt.plot(epochs, mean_pe_values, marker='o', linestyle='-', color='b', label='mean_PE')
plt.fill_between(epochs, mean_pe_values - std_pe_values, mean_pe_values + std_pe_values, color='blue', alpha=0.3, label='std_dev_PE')

plt.title('2mev: PE vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('PE')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
model.load_weights('/root/shuju/fangan72mev_average_weights.h5')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
import pandas as pd

y_pred_test = mm.inverse_transform(model.predict(datax_test))
y_val_test = mm.inverse_transform(datay_test)

data = {
    '真实值': y_val_test.ravel(),
    '预测值': y_pred_test.ravel()       
    
}

df = pd.DataFrame(data)
output_file = '/root/shuju/fangan3bijiao2mev.csv'  
df.to_csv(output_file, index=False)

print(f'预测值和真实值已保存到 {output_file}')

print("预测值：")
print(y_pred_test)
print("真实值：")
print(y_val_test)


In [ ]:
import pandas as pd

input_file1 = '/root/shuju/ceshiji.csv'  
df1 = pd.read_csv(input_file1)


input_file2 = '/root/shuju/fangan3bijiao2mev.csv'  
df2 = pd.read_csv(input_file2)

column1 = df2.iloc[:, 0]
column2 = df2.iloc[:, 1]

df2['真实值'] = column1  
df2['预测值'] = column2  

column3 = df2.iloc[:, :]  
merged_df = pd.concat([df1.iloc[:, 0], column3], axis=1)
output_file = '/root/shuju/fangan3bijiao2mev.csv'  
merged_df.to_csv(output_file, index=False)

print(f'数据已整合到 {output_file}')


In [ ]:
import pandas as pd

df_original = pd.read_csv('/root/shuju/ceshiji.csv')
dates_filtered = df_original[df_original['2mev_next'] >= 3.5]['Formatted Date']
df_comparison = pd.read_csv('/root/shuju/fangan3bijiao2mev.csv')
df_filtered = df_comparison[df_comparison['Formatted Date'].isin(dates_filtered)]
df_filtered.to_csv('/root/shuju/filtered_data_based_on_date.csv', index=False)
